TensowFlowのバージョンを指定（最新のバージョンにすることを推奨）

In [ ]:
!pip uninstall -y tensorflow
!pip install tensorflow==2.15.0

# パターン認識

## データの読み込み

### データの読み込み

In [ ]:
from pandas._libs.tslibs.vectorized import normalize_i8_timestamps
import io
import pandas as pd
import csv
import time

start = time.perf_counter()

# データのパスを指定
#リアルデータのみ
f2 = "/content/drive/MyDrive/Colab Notebooks/tokken/deeplerning/analisys/data(hitByHit)/run13137_0_1500000_PrintDCHit_processed_sort_corDrift.csv"
#結合型track数2のデータ
f3 = "/content/drive/MyDrive/Colab Notebooks/tokken/deeplerning/analisys/data(hitByHit)/run13690_0_1500000_PrintDCHit_TRK1_combineToTRK2_sort.csv"
f4 = "/content/drive/MyDrive/Colab Notebooks/tokken/deeplerning/analisys/data(hitByHit)/run12230_0_1500000_PrintDCHit_TRK1_combineToTRK2_sort.csv"

col_names = [ 'c{0:02d}'.format(i) for i in range(302) ]


auto = pd.read_csv(f2,names=col_names, dtype="float32")
### 結合が必要な場合
auto3 = pd.read_csv(f3,names=col_names, dtype="float32")
auto4 = pd.read_csv(f4,names=col_names, dtype="float32")
auto2 = pd.concat([auto3, auto4])
auto2['c301'] = 2
print(auto2)
auto = pd.concat([auto, auto2])
del f2,f3,f4, col_names, auto2, auto3, auto4
###

print(time.perf_counter() - start)

print(auto)

# c301列をc302列にコピー
auto['c302'] = auto['c301'].copy()


# c301列で0以外の値を持つ行をすべて1に変更
auto.loc[auto['c301'] != 0, 'c301'] = 1

print(auto)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 正規化

In [ ]:
import numpy as np

# 正規化関数
def normalize_column(column, min_val, max_val):
    return (column - min_val) / (max_val - min_val)

# 正規化の範囲を定義
ranges = [(0, 1074), (-637, 637), (-2, 15)]

# 正規化を適用
for i in range(1, 301):
    col_name = 'c{0:02d}'.format(i)
    range_index = (i - 1) % 3
    min_val, max_val = ranges[range_index]
    auto[col_name] = normalize_column(auto[col_name], min_val, max_val)

# 結果を表示
print(auto)

### データのクラス割合を調整

In [ ]:
import pandas as pd
from sklearn.utils import resample


print(f"総イベント数{len(auto)}")

# 0と1のデータをそれぞれ抽出
data_0 = auto[auto['c302'] == 0]
data_1 = auto[auto['c302'] == 1]
data_2 = auto[auto['c302'] == 2]

# データ数を取得
count_0 = len(data_0)
count0_1 = len(data_1)
count_2 = len(data_2)

# track有りデータのtrack数1,2の割合を等しくする。
target_count_1_2 = min(50000, min(count0_1, count_2))
sampled0_data_1 = resample(data_1, replace=False, n_samples=50000, random_state=42)
sampled0_data_2 = resample(data_2, replace=False, n_samples=50000, random_state=42)
sampled0_count_1 = len(sampled0_data_1)
sampled0_count_2 = len(sampled0_data_2)
# 新しいデータフレームに結合
data_1 = pd.concat([sampled0_data_1, sampled0_data_2])

print(f'Sampled 1s: {sampled0_count_1}, Sampled 2s: {sampled0_count_2}')

del sampled0_data_1, sampled0_data_2,sampled0_count_1,sampled0_count_2,data_2,target_count_1_2

count_1 = len(data_1)

# サンプリング後の目標データ数を設定
target_count_0_1 = min(100000, min(count_0, count_1))  # 最大200,000になるように調整

# resample関数を使ってサンプリング
sampled_data_0 = resample(data_0, replace=False, n_samples=target_count_0_1, random_state=42)
sampled_data_1 = resample(data_1, replace=False, n_samples=target_count_0_1, random_state=42)
sampled_count_0 = len(sampled_data_0)
sampled_count_1 = len(sampled_data_1)

# 新しいデータフレームに結合
auto = pd.concat([sampled_data_0, sampled_data_1])

del data_0, data_1, sampled_data_0, sampled_data_1

# 割合を表示
print(f'Original class 0: {count_0}, Original class 1: {count0_1}, Original class 2: {count_2}')
print(f'Sampled 0s: {sampled_count_0}, Sampled 1s: {sampled_count_1}')

del sampled_count_0, sampled_count_1,count_0,count0_1,count_2,target_count_0_1



### 入力層に取り込む列のラベルのリストを作成

In [ ]:
col_names = [col for col in auto.columns if 'c' in col]
col_names.remove('c00')
col_names.remove('c301')
col_names.remove('c302')
print(col_names)

## tensowflowで行う機械学習

### パッケージ

In [ ]:
import sys
sys.setrecursionlimit(2000)
import numpy as np

from __future__ import print_function

from matplotlib import pyplot as plt
import tensorflow as tf
print(tf.__version__)
from tensorflow.python.client import device_lib

# At KEK
import tensorflow.keras as kera
#
from tensorflow.keras import backend
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from keras.optimizers import SGD

from sklearn.model_selection import train_test_split
from hyperopt import Trials
import hyperopt
from hyperopt import fmin, tpe, hp
from sklearn.metrics import f1_score, recall_score, precision_score
from keras.callbacks import LearningRateScheduler
from datetime import datetime

### Sigmoidを用いた二値分類

#### 学習と評価

In [10]:

epochs = 20

X = auto[col_names] # すべての列を扱う
y = auto['c301']


c00_info = auto['c00'] # 列"c00"（イベント番号）を別の変数に保存
c302_info = auto['c302'] #  列"c302"（トラック数）を別の変数に保存

# 訓練データとテストデータに分ける
x_train, x_test, y_train, y_test, c00_train, c00_test, c302_train, c302_test = train_test_split(X, y, c00_info, c302_info, test_size=0.3, random_state=0)

del auto,X,y,c00_info

#### ハイパーパラメータ探索

In [ ]:
start = time.perf_counter()

input_size = 300
output_size = 1
threshold = 0.5

# F1スコアを保存するためのリストを作成
f1_scores = []

# Step 1: ハイパーパラメータ探索範囲を定義
space = {
    'batch_size': hp.choice('batch_size', [64, 128, 256, 512]), #バッチサイズ
    'layers': hp.randint('layers', 20) + 5,  # 中間層
    'neurons': hp.choice('neurons', [64, 128, 256, 512,1024]), #ニューロン数
    'learning_rate': hp.loguniform('learning_rate', -4, -2),
    # 'threshold': hp.uniform('threshold', 0.5, 0.9)
}

# Step 2: 目的関数を定義
def objective(params):
    batch_size = params['batch_size']
    layers = params['layers']
    neurons = params['neurons']
    learning_rate = params['learning_rate']

    # class_weights = {0: 1.0, 1: 2.0}  # 重みを調整

    # モデルの構築と訓練
    model = Sequential()
    model.add(Dense(input_size, activation='relu', input_shape=(input_size,)))
    model.add(Dropout(0.2))
    for i in range(1, layers-1):
        model.add(Dense(neurons, activation='relu'))
        model.add(Dropout(0.2))
    model.add(Dense(output_size, activation='sigmoid'))

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                  optimizer=kera.optimizers.Adam(learning_rate),
                  metrics=['accuracy'])




    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0,
                        validation_data=(x_test, y_test),
                        # class_weight=class_weights #重みづけ
    )

    # F1スコアを評価指標
    y_pred = model.predict(x_test)
    # 予測されたクラスを取得（確率が0.5以上の場合は1、それ以外の場合は0としています）
    y_pred_classes = np.where(y_pred >= threshold, 1, 0)

    # 指標の計算
    f1 = f1_score(y_test, y_pred_classes) # F1スコアを計算

    f1_scores.append(f1)
    print(f"Current parameters: {params}")
    print(f"Current F1 score: {f1}")

    return -f1  # 目的関数は最大化する必要があるため、負のF1スコアを返す


# Step 3: hyperoptを使用して最適なハイパーパラメータを探索
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=1)  # 30回評価
print("Best Hyperparameters:", best)

# F1スコアを図にする
plt.plot(f1_scores)
plt.xlabel('Evaluation number')
plt.ylabel('F1 score')
plt.show()

print(time.perf_counter() - start)

#### 学習

In [ ]:
# 2024/9/18物理学会でのセッティング

start = time.perf_counter()

# GPUの非決定的最適化を回避して再現性の確保
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()
###

input_size = 300
output_size = 1

batch_size = 512
epochs = 50
layers = 3
neurons = 64
learning_rate = 0.001
threshold = 0.23

# class_weights = {0: 1.0, 1: 2.0}  # class1の重みを調整

model = Sequential()

# model.add(InputLayer(input_shape=(200,)))
model.add(Dense(input_size, activation='relu', input_shape=(input_size,)))
model.add(Dropout(0.2))

# middle layer 2 to layer 500
for i in range(1, layers-1, 1):
    model.add(Dense(neurons, activation='relu'))
    model.add(Dropout(0.2))

model.add(Dense(output_size, activation='sigmoid'))

model.summary()

def track_acc(self, x, y_true, logs):
    # テストデータに対する予測を取得
    y_pred = self.model.predict(x)
    # 予測されたクラスを取得（確率が0.5以上の場合は1、それ以外の場合は0としています）
    y_pred_classes = np.where(y_pred >= threshold, 1, 0)

    # 指標の計算
    f1 = f1_score(y_true, y_pred_classes)
    recall = recall_score(y_true, y_pred_classes)
    precision = precision_score(y_true, y_pred_classes)

    del y_pred, y_pred_classes


    return {'f1': f1, 'recall': recall, 'precision': precision}

class ElementwiseAccuracy(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        test_acc = track_acc(self, x_test, y_test, logs)
        train_acc = track_acc(self, x_train, y_train, logs)
        logs['test_f1'] = test_acc['f1']
        logs['train_f1'] = train_acc['f1']
        logs['test_recall'] = test_acc['recall']
        logs['train_recall'] = train_acc['recall']
        logs['test_precision'] = test_acc['precision']
        logs['train_precision'] = train_acc['precision']

# カスタムコールバックを定義
accuracy_callback = ElementwiseAccuracy()



# モデルのコンパイル時にコールバックを指定
# model.compile(loss='categorical_crossentropy',
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=kera.optimizers.Adam(learning_rate),
              metrics=['accuracy'])

# モデルの訓練時にコールバックを指定
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[accuracy_callback],
                    # class_weight=class_weights,
)

# モデルの保存
# 現在の日時を取得
now = datetime.now()
# 日時を文字列に変換（例: '2024-05-29_14-12-02'）
timestamp_str = now.strftime("%Y%m%d%H%M%S")
# ファイル名を作成（例: 'my_model_2024-05-29_14-12-02.h5'）
filename = timestamp_str + 'allDataNoiseReduction.h5'
# モデルの保存
# model.save("/content/drive/MyDrive/Colab Notebooks/tokken/deeplerning/analisys/model/" + filename)
print(filename)

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# （検証データ）出力の確率分布の表示のための整形
y_pred_array = np.array(model.predict(x_test))
y_test_array = np.array(y_test)
c302_test_array = np.array(c302_test)
y_pred_0 = y_pred_array[y_test_array == 0]
y_pred_1 = y_pred_array[y_test_array == 1]
# トラック数ごとで正答率を確認するため
y_pred_track = [y_pred_array[c302_test_array == i] for i in range(6)]

# （学習データ）出力の確率分布の表示のための整形
y_train_pred_array = np.array(model.predict(x_train))
y_train_array = np.array(y_train)
c302_train_array = np.array(c302_train)
y_train_pred_0 = y_train_pred_array[y_train_array == 0]
y_train_pred_1 = y_train_pred_array[y_train_array == 1]
# トラック数ごとで正答率を確認するため
y_train_pred_track = [y_train_pred_array[c302_train_array == i] for i in range(6)]


# F1スコアを計算
y_pred_classes = np.where(y_pred_array >= threshold, 1, 0)
f1 = f1_score(y_test, y_pred_classes)
print("f1_score",f1)


def plot_history(train_history, test_history, title, train_label, test_label):
  plt.figure(figsize=(12, 6))
  plt.plot(train_history, label=train_label)
  plt.plot(test_history, label=test_label)
  plt.title(title)
  plt.ylabel(title)
  plt.xlabel('Epoch')
  plt.legend()
  plt.show()

# Plot loss
plot_history(history.history['loss'], history.history['val_loss'], 'loss', 'train_loss', 'test_loss')

# Plot accuracy
plot_history(history.history['accuracy'], history.history['val_accuracy'], 'accuracy', 'train_accuracy', 'test_accuracy')

# Plot f1
plot_history(history.history['train_f1'], history.history['test_f1'], 'f1', 'train_f1', 'test_f1')

# Plot recall
plot_history(history.history['train_recall'], history.history['test_recall'], 'recall', 'train_recall', 'test_recall')

# Plot precision
plot_history(history.history['train_precision'], history.history['test_precision'], 'precision', 'train_precision', 'test_precision')

n_bin = 100
x_max = 1.0
x_min = 0.0
bins = np.linspace(x_min, x_max, n_bin)

#（検証データ）出力の確率分布を表示
print("検証データ")
plt.hist(y_pred_0, bins=bins, alpha=0.5, color='blue', label='class 0')
plt.hist(y_pred_1, bins=bins, alpha=0.5, color='red', label='class 1')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.title('Probability Distribution')
# plt.axvline(x=threshold, color='black', linestyle='--')
plt.legend()
plt.show()

plt.hist(y_pred_0, bins=bins, alpha=0.5, color='blue', label='class 0')
plt.hist(y_pred_1, bins=bins, alpha=0.5, color='red', label='class 1')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.title('Probability Distribution')
# plt.axvline(x=threshold, color='black', linestyle='--')
plt.yscale('log')
plt.legend()
plt.show()

print("y_pred_0 の要素数:", len(y_pred_0))
print("y_pred_1 の要素数:", len(y_pred_1))
print("y_pred_0 > threshold の要素数:", len(y_pred_0[y_pred_0 > threshold] == "true"))
print("y_pred_1 > threshold の要素数:", len(y_pred_1[y_pred_1 > threshold] == "true"))


#（学習データ）出力の確率分布を表示
print("学習データ")
plt.hist(y_train_pred_0, bins=bins, alpha=0.5, color='blue', label='class 0')
plt.hist(y_train_pred_1, bins=bins, alpha=0.5, color='red', label='class 1')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.title('Probability Distribution')
# plt.axvline(x=threshold, color='black', linestyle='--')
plt.legend()
plt.show()

plt.hist(y_train_pred_0, bins=bins, alpha=0.5, color='blue', label='class 0')
plt.hist(y_train_pred_1, bins=bins, alpha=0.5, color='red', label='class 1')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.title('Probability Distribution')
# plt.axvline(x=threshold, color='black', linestyle='--')
plt.yscale('log')
plt.legend()
plt.show()

print("y_train_pred_0 の要素数:", len(y_train_pred_0))
print("y_train_pred_1 の要素数:", len(y_train_pred_1))
print("y_train_pred_0 > threshold の要素数:", len(y_train_pred_0[y_train_pred_0 > threshold] == "true"))
print("y_train_pred_1 > threshold の要素数:", len(y_train_pred_1[y_train_pred_1 > threshold] == "true"))


print(time.perf_counter() - start)

#### thresholdを動かして結果を確認

In [ ]:
threshold = 0.23
threshold1 = 0.5


#出力の確率分布を表示
def plot_ditribution(data_0, data_1, title, class_label_0, class_label_1):
  plt.hist(data_0, bins=200, alpha=0.5, color='blue', label=class_label_0)
  plt.hist(data_1, bins=200, alpha=0.5, color='red', label=class_label_1)
  plt.xlabel('Predicted Probability')
  plt.ylabel('Frequency')
  plt.title(title)
  plt.axvline(x=threshold1, color='black', linestyle='--')
  plt.axvline(x=threshold, color='black', linestyle='--')
  # plt.yscale('log')
  plt.legend()
  plt.show()

  plt.hist(data_0, bins=200, alpha=0.5, color='blue', label=class_label_0)
  plt.hist(data_1, bins=200, alpha=0.5, color='red', label=class_label_1)
  plt.xlabel('Predicted Probability')
  plt.ylabel('Frequency')
  plt.title(title)
  plt.axvline(x=threshold1, color='black', linestyle='--')
  plt.axvline(x=threshold, color='black', linestyle='--')
  plt.yscale('log')
  plt.legend()
  plt.show()

# 検証データの
print("検証データ")
plot_ditribution(y_pred_track[0], y_pred_track[1], 'Probability Distribution', 'class 0', 'class1')
plot_ditribution(y_pred_track[0], y_pred_track[2], 'Probability Distribution', 'class 0', 'class2')
plot_ditribution(y_pred_track[0], y_pred_track[3], 'Probability Distribution', 'class 0', 'class3')
plot_ditribution(y_pred_track[0], y_pred_track[4], 'Probability Distribution', 'class 0', 'class4')
plot_ditribution(y_pred_track[0], y_pred_track[5], 'Probability Distribution', 'class 0', 'class5')

#出力の確率分布を表示
plt.hist(y_pred_0, bins=200, alpha=0.5, color='blue', label='class 0')
plt.hist(y_pred_1, bins=200, alpha=0.5, color='red', label='class 1')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.title('Probability Distribution')
plt.axvline(x=threshold, color='black', linestyle='--')
plt.legend()
plt.show()
#出力の確率分布を表示
plt.hist(y_pred_0, bins=200, alpha=0.5, color='blue', label='class 0')
plt.hist(y_pred_1, bins=200, alpha=0.5, color='red', label='class 1')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.title('Probability Distribution')
plt.axvline(x=threshold, color='black', linestyle='--')
plt.yscale('log')
plt.legend()
plt.show()

print("y_pred_0 の要素数:", len(y_pred_0))
print("y_pred_1 の要素数:", len(y_pred_1))
print(f'y_pred_0> {threshold} の割合:', len(y_pred_0[y_pred_0 > threshold] == "true") / len(y_pred_0))
print(f"y_pred_1> {threshold} の割合:", len(y_pred_1[y_pred_1 > threshold] == "true") / len(y_pred_1))

print(f'y_pred_0> {threshold1} の割合:', len(y_pred_0[y_pred_0 > threshold1] == "true") / len(y_pred_0))
print(f"y_pred_1> {threshold1} の割合:", len(y_pred_1[y_pred_1 > threshold1] == "true") / len(y_pred_1))

thresholds = [threshold, threshold1]
for i in range(6):
    print(f"y_pred_track[{i}] の要素数:", len(y_pred_track[i]))
    for t in thresholds:
        print(f'y_pred_track[{i}]> {t} の割合:', len(y_pred_track[i][y_pred_track[i] > t] == "true") / len(y_pred_track[i]))

# 学習データの予測結果の詳細
print("学習データ")

# モデルを読み込んで検証

## データの読み込み

### パッケージ

In [ ]:
import sys
sys.setrecursionlimit(2000)
import numpy as np

from __future__ import print_function

from matplotlib import pyplot as plt
import tensorflow as tf
print(tf.__version__)
from tensorflow.python.client import device_lib

# At KEK
import tensorflow.keras as kera
#
from tensorflow.keras import backend
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from keras.optimizers import SGD

from sklearn.model_selection import train_test_split
from hyperopt import Trials
import hyperopt
from hyperopt import fmin, tpe, hp
from sklearn.metrics import f1_score, recall_score, precision_score
from keras.callbacks import LearningRateScheduler
from datetime import datetime
from tensorflow.keras.models import load_model

### データの読み込み

In [ ]:
from pandas._libs.tslibs.vectorized import normalize_i8_timestamps
import io
import pandas as pd
import csv
import time

start = time.perf_counter()

# リモート
#リアルデータのみ
f2 = "/content/drive/MyDrive/（あなたのパス）/run12230_0_1500000_PrintDCHit_processed_corDrift_sort.csv"
# f2 = "/content/drive/MyDrive/（あなたのパス）/run12230_0_1500000_PrintDCHit_TRK1_combineToTRK2_sort.csv"
# f2 = "/content/drive/MyDrive/（あなたのパス）/run12230_0_1500000_PrintDCHit_TRK0-1_combineToTRK1_sort.csv"


col_names = [ 'c{0:02d}'.format(i) for i in range(302) ]


# 結合が必要な場合
# auto1 = pd.read_csv(f1,names=col_names)
# auto2 = pd.read_csv(f2,names=col_names)

# auto = pd.concat([auto1, auto2]) # 結合

auto = pd.read_csv(f2,names=col_names, dtype="float32")

del f2, col_names

print(time.perf_counter() - start)

print(auto)

# c301列をc302列にコピー
auto['c302'] = auto['c301'].copy()

# c401列で0以外の値を持つ行をすべて1に変更
auto.loc[auto['c301'] != 0, 'c301'] = 1

print(auto)


### 入力層に取り込む列のラベルのリストを作成

In [ ]:
col_names = [col for col in auto.columns if 'c' in col]
col_names.remove('c00')
col_names.remove('c301')
col_names.remove('c302')
print(col_names)

### 検証

In [ ]:
x = auto[col_names] # すべての列を扱う
y = auto['c301']
c302 = auto['c302']

In [ ]:
threshold = 0.5
#モデルの読み込み
# model = load_model("/content/drive/MyDrive/（あなたの作成したモデルのパス）")

# 出力の確率分布の表示のための整形
y_pred_array = np.array(model.predict(x))
y_test_array = np.array(y)
c302_test_array = np.array(c302)

y_pred_0 = y_pred_array[y_test_array == 0]
y_pred_1 = y_pred_array[y_test_array == 1]

# トラック数ごとで正答率を確認するため
y_pred_track = [y_pred_array[c302_test_array == i] for i in range(6)]

# F1スコアを計算
y_pred_classes = np.where(y_pred_array >= threshold, 1, 0)
f1 = f1_score(y, y_pred_classes)
print("f1_score",f1)

In [ ]:
threshold = 0.23
threshold1 = 0.5

max_value_axis0 = np.max(y_pred_array, axis=0)
print("各行の最大値:", max_value_axis0)#各行の最大値: [0.7297788]

#出力の確率分布を表示
def plot_ditribution(data_0, data_1, title, class_label_0, class_label_1):
  plt.hist(data_0, bins=300, alpha=0.5, color='blue', label=class_label_0)
  plt.hist(data_1, bins=300, alpha=0.5, color='red', label=class_label_1)
  plt.xlabel('Predicted Probability')
  plt.ylabel('Frequency')
  plt.xlim(0.0, 1.0)
  plt.title(title)
  # plt.axvline(x=threshold1, color='black', linestyle='--')
  plt.axvline(x=threshold, color='black', linestyle='--')
  plt.legend()
  plt.show()
  plt.hist(data_0, bins=300, alpha=0.5, color='blue', label=class_label_0)
  plt.hist(data_1, bins=300, alpha=0.5, color='red', label=class_label_1)
  plt.xlabel('Predicted Probability')
  plt.ylabel('Frequency')
  plt.xlim(0.0, 1.0)
  plt.title(title)
  # plt.axvline(x=threshold1, color='black', linestyle='--')
  plt.axvline(x=threshold, color='black', linestyle='--')
  plt.yscale('log')
  plt.legend()
  plt.show()

plot_ditribution(y_pred_track[0], y_pred_track[1], 'Probability Distribution', 'class 0', 'class1')
plot_ditribution(y_pred_track[0], y_pred_track[2], 'Probability Distribution', 'class 0', 'class2')
plot_ditribution(y_pred_track[0], y_pred_track[3], 'Probability Distribution', 'class 0', 'class3')
plot_ditribution(y_pred_track[0], y_pred_track[4], 'Probability Distribution', 'class 0', 'class4')
plot_ditribution(y_pred_track[0], y_pred_track[5], 'Probability Distribution', 'class 0', 'class5')

plt.hist(y_pred_0, bins=300, alpha=0.5, color='blue', label='class 0')
plt.hist(y_pred_1, bins=300, alpha=0.5, color='red', label='class 1')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.xlim(0.0, 1.0)
plt.title('Probability Distribution')
# plt.axvline(x=threshold1, color='black', linestyle='--')
plt.axvline(x=threshold, color='black', linestyle='--')
# plt.yscale('log')
plt.legend()
plt.show()
plt.hist(y_pred_0, bins=300, alpha=0.5, color='blue', label='class 0')
plt.hist(y_pred_1, bins=300, alpha=0.5, color='red', label='class 1')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.xlim(0.0, 1.0)
plt.title('Probability Distribution')
# plt.axvline(x=threshold1, color='black', linestyle='--')
plt.axvline(x=threshold, color='black', linestyle='--')
plt.yscale('log')
plt.legend()
plt.show()

print("y_pred_0 の要素数:", len(y_pred_0))
print("y_pred_1 の要素数:", len(y_pred_1))
# print(f'y_pred_0> {threshold} の割合:', len(y_pred_0[y_pred_0 > threshold] == "true") / len(y_pred_0))
print(f"y_pred_1> {threshold} の割合:", len(y_pred_1[y_pred_1 > threshold] == "true") / len(y_pred_1))

# print(f'y_pred_0> {threshold1} の割合:', len(y_pred_0[y_pred_0 > threshold1] == "true") / len(y_pred_0))
print(f"y_pred_1> {threshold1} の割合:", len(y_pred_1[y_pred_1 > threshold1] == "true") / len(y_pred_1))

thresholds = [threshold, threshold1]
for i in range(6):
    print(f"y_pred_track[{i}] の要素数:", len(y_pred_track[i]))
    for t in thresholds:
        print(f'y_pred_track[{i}]> {t} の割合:', len(y_pred_track[i][y_pred_track[i] > t] == "true") / len(y_pred_track[i]))